### Reference: Topic modeling with spaCy & scikit-learn LDA
#### https://www.kaggle.com/thebrownviking20/topic-modelling-with-spacy-and-scikit-learn/notebook

In [40]:
# regular imports
import numpy as np
import pandas as pd
import os
import time
import string
import matplotlib.pyplot as plt

from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# imports for scikit-learn & LDA
import sklearn
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.manifold import TSNE
import concurrent.futures

# imports for scikit-learn & LDA
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig

# Plotly based imports for visualization
from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

# spaCy based imports
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

In [2]:
# Load the spacy model installed (using the medium model)
nlp = spacy.load('en_core_web_md')

# Define the working directory & raw input datasets
rel_path = './'
infile = '050319_acled_all.csv'

# Define the directory for saving LDA visualizations as HTML files
lda_vis_path = './lda_vis/'

# Read in the raw file
df = pd.read_csv(os.path.join(rel_path, infile))

# Define the punctuations & stop words
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

In [5]:
df.notes[3]

'27 April. Rioters blocked a road in Save to protest the legislative election. Police called for reinforcements to control the situation. Injuries on both sides were reported. [size=no report]'

#### Named Entity Recognition

In [7]:
doc = nlp(df["notes"][5])
spacy.displacy.render(doc, style='ent',jupyter=True)

#### Lemmatization

In [8]:
# Lemmatization
review = str(" ".join([i.lemma_ for i in doc]))
doc = nlp(review)
spacy.displacy.render(doc, style='ent',jupyter=True)

#### Part-of-Speech Tagging

In [9]:
for i in nlp(review):
    print(i,"=>",i.pos_)

27 => NUM
April => PROPN
. => PUNCT
unknown => ADJ
actor => NOUN
destroy => VERB
election => NOUN
material => NOUN
of => ADP
the => DET
Autonomous => PROPN
National => PROPN
Electoral => PROPN
Commission => PROPN
( => PUNCT
CENA => PROPN
) => PUNCT
in => ADP
Akpadanou => PROPN
ahead => ADV
of => ADP
the => DET
legislative => ADJ
election => NOUN
. => PUNCT


#### Parser function to Tokenize conflict notes

In [16]:
# Parser for conlfict notes
parser = English()
def spacy_tokenizer(note):
    mytokens = parser(str(note))
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [15]:
conflict = df["notes"][5]
print('\n------ Raw Note ------\n', conflict)
print('\n---- Parsed Note ----\n', spacy_tokenizer(conflict))


------ Raw Note ------
 27 April. Unknown actors destroyed election materials of the Autonomous National Electoral Commission (CENA) in Akpadanou ahead of the legislative elections.

---- Parsed Note ----
 ['27', 'april', 'unknown', 'actor', 'destroy', 'election', 'material', 'autonomous', 'national', 'electoral', 'commission', 'cena', 'akpadanou', 'ahead', 'legislative', 'election']


In [17]:
# Tokenizing the entire dataset
tqdm.pandas()
df["processed_notes"] = df["notes"].progress_apply(spacy_tokenizer)

100%|██████████| 509157/509157 [03:43<00:00, 2279.10it/s]


### Topic Modeling

In [18]:
# Creating a vectorizer
vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(df["processed_notes"])

<>:2: DeprecationWarning:

invalid escape sequence \-

<>:2: DeprecationWarning:

invalid escape sequence \-

<>:2: DeprecationWarning:

invalid escape sequence \-

<ipython-input-18-72af7956ffb4>:2: DeprecationWarning:

invalid escape sequence \-



In [19]:
# Number of Topics/Clusters to create
NUM_TOPICS = 10

In [20]:
%%time

# Running the LDA model
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10
Wall time: 24min 56s


In [38]:
%%time

# Running the LDA model
my_lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=2, learning_method='online')

Wall time: 0 ns


In [45]:
# data_lda = lda.fit_transform(data_vectorized)
# type(my_lda)
# type(vectorizer)
data_vectorized = vectorizer.fit_transform(df["processed_notes"])

In [51]:
# type(data_vectorized)
# len(vectorizer.get_feature_names())
NUM_TOPICS

10

In [52]:
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [54]:
%%time
lda.fit(data_vectorized)

Wall time: 24min 3s


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_components=10, n_jobs=None, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [55]:
# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]])

In [56]:
# Keywords for topics clustered by Latent Dirichlet Allocation
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('bomb', 20168.683397616383), ('area', 14531.997497493341), ('house', 13101.151029355993), ('road', 12364.8489672896), ('injure', 11830.987120257585), ('civilian', 11539.908587226944), ('hit', 10600.476925249102), ('kill', 10197.848091250584), ('vehicle', 10168.767103843222), ('july', 9973.211162890988)]
Topic 1:
[('force', 20587.2032216591), ('report', 13423.041634953355), ('pro', 12840.003598229767), ('reportedly', 12084.925447286527), ('area', 11605.393663322573), ('casualty', 11008.828682537158), ('border', 10560.292201913864), ('district', 9681.694719133086), ('houthi', 8178.195460529578), ('governorate', 7035.943598418387)]
Topic 2:
[('unknown', 11306.268735330885), ('weapon', 8494.757551003835), ('explosion', 7340.348712778293), ('near', 5924.709303393687), ('total', 5514.758487750639), ('involve', 5138.005005686441), ('saturday', 4673.23109813958), ('tfg', 4424.344366672602), ('observe', 3538.550251412824), ('israeli', 2987.94235392474)]
Topic 3:
[('rebel',

In [57]:
%%time
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
6      135.731613  297.573364       1        1  19.516104
9      382.902283 -133.121124       2        1  15.834622
5       50.524933  103.428993       3        1  10.201824
0       -1.992439  -95.287178       4        1  10.014948
8     -131.401871  213.223465       5        1   9.554017
7      159.696609 -273.887756       6        1   9.361443
1     -205.424545  -19.633326       7        1   7.867996
4     -100.382881 -279.601868       8        1   7.625384
3      322.661011  137.885544       9        1   6.622578
2      196.076675  -41.287067      10        1   3.401084, topic_info=      Category           Freq         Term          Total  loglift  logprob
28702  Default  119875.000000      protest  119875.000000  30.0000  30.0000
30065  Default   58129.000000        rebel   58129.000000  29.0000  29.0000
28184  Default   76055.000000       police   76055.000000  28.0000  28.0000
19037  Default  141199.000000         kill  141199.000000  27.0000  27.0000
23392  Default   45936.000000     militant   45936.000000  26.0000  26.0000
11749  Default  125495.000000        force  125495.000000  25.0000  25.0000
37535  Default   38129.000000      unknown   38129.000000  24.0000  24.0000
30361  Default   89470.000000       report   89470.000000  23.0000  23.0000
15670  Default   39285.000000       injury   39285.000000  22.0000  22.0000
2781   Default   77362.000000       attack   77362.000000  21.0000  21.0000
23399  Default   46167.000000     military   46167.000000  20.0000  20.0000
11316  Default   48858.000000     fatality   48858.000000  19.0000  19.0000
13089  Default   46546.000000   government   46546.000000  18.0000  18.0000
25451  Default   40381.000000         near   40381.000000  17.0000  17.0000
34424  Default   45109.000000        stage   45109.000000  16.0000  16.0000
38640  Default   18183.000000       weapon   18183.000000  15.0000  15.0000
28727  Default   26902.000000     province   26902.000000  14.0000  14.0000
28584  Default   22862.000000          pro   22862.000000  13.0000  13.0000
30365  Default   21517.000000   reportedly   21517.000000  12.0000  12.0000
9630   Default   64043.000000     district   64043.000000  11.0000  11.0000
10979  Default   15712.000000    explosion   15712.000000  10.0000  10.0000
7344   Default   19282.000000    coalition   19282.000000   9.0000   9.0000
6406   Default   27699.000000     casualty   27699.000000   8.0000   8.0000
8841   Default   31131.000000       demand   31131.000000   7.0000   7.0000
30184  Default   21055.000000       regime   21055.000000   6.0000   6.0000
27411  Default   18817.000000        party   18817.000000   5.0000   5.0000
5379   Default   18803.000000       border   18803.000000   4.0000   4.0000
36876  Default   17561.000000        troop   17561.000000   3.0000   3.0000
2239   Default   76673.000000         area   76673.000000   2.0000   2.0000
33119  Default   24199.000000        shell   24199.000000   1.0000   1.0000
...        ...            ...          ...            ...      ...      ...
16040  Topic10    6395.542232      israeli    6396.741597   3.3809  -3.7928
27126  Topic10    5169.452632  palestinian    5170.651991   3.3808  -4.0056
33356  Topic10    3508.765067         shot    3509.964473   3.3807  -4.3931
33891  Topic10    3457.948607          smm    3459.147947   3.3807  -4.4077
26765  Topic10    3386.596767         osce    3387.796107   3.3807  -4.4286
1349   Topic10    3208.827656     algerian    3210.027123   3.3807  -4.4825
38932  Topic10    2665.190327          www    2666.389721   3.3806  -4.6681
9464   Topic10    2281.696166    direction    2282.895564   3.3806  -4.8235
12382  Topic10    2230.184643         gaza    2231.384005   3.3805  -4.8463
11049  Topic10    2157.073694          faa    2158.273157   3.3805  -4.8797
30272  Topic10    2053.548599       relief    2054.748061   3.3805  -

In [60]:
vectorizer

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.9, max_features=None, min_df=5,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='[a-zA-Z\\-][a-zA-Z\\-]{2,}',
        tokenizer=None, vocabulary=None)

In [ ]:
df.notes.head(5)

In [64]:
type(dash)

pyLDAvis._prepare.PreparedData

#### Save the Topic Modeling Visualization created as an HTML file

In [77]:
lda_vis_path = './lda_vis/'
out_vis_file = 'full_data_10_topics.html'

In [78]:
pyLDAvis.save_html(dash, fileobj=os.path.join(lda_vis_path, out_vis_file))

In [114]:
data_vectorized.shape

(509157, 39799)

In [94]:
%%time

check = lda.transform(data_vectorized)

Wall time: 1min 22s


In [105]:
lda.transform(data_vectorized[5])

array([[0.07986111, 0.00714286, 0.00714732, 0.16573761, 0.63212936,
        0.00714328, 0.00714301, 0.00714286, 0.0794094 , 0.00714319]])

In [106]:
type(df["processed_notes"])

pandas.core.series.Series

In [112]:
type(lda.transform(data_vectorized[5])[0])

numpy.ndarray

In [113]:
pyLDAvis.sklearn.prepare(lda, data_vectorized[:1000], vectorizer, mds='tsne')

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
6      -83.612099  -13.499725       1        1  18.071591
7      100.056252 -161.636581       2        1  13.783197
3     -187.829086  -81.746361       3        1  10.259316
1       13.422092   59.958382       4        1  10.233603
8      150.771957   -7.656671       5        1   9.193099
5      112.426636  136.136444       6        1   8.822521
9     -166.099808   83.400238       7        1   8.262423
2      -60.776428 -161.835922       8        1   8.173202
4       28.624241  -61.006283       9        1   6.665057
0      -50.690342  169.438507      10        1   6.535991, topic_info=      Category        Freq          Term       Total  loglift  logprob
30065  Default  189.000000         rebel  189.000000  30.0000  30.0000
28702  Default  233.000000       protest  233.000000  29.0000  29.0000
37535  Default  167.000000       unknown  167.000000  28.0000  28.0000
28184  Default  144.000000        police  144.000000  27.0000  27.0000
15670  Default  121.000000        injury  121.000000  26.0000  26.0000
38640  Default   91.000000        weapon   91.000000  25.0000  25.0000
19037  Default  247.000000          kill  247.000000  24.0000  24.0000
11749  Default  317.000000         force  317.000000  23.0000  23.0000
23392  Default   83.000000      militant   83.000000  22.0000  22.0000
10979  Default   79.000000     explosion   79.000000  21.0000  21.0000
30361  Default  237.000000        report  237.000000  20.0000  20.0000
11316  Default  148.000000      fatality  148.000000  19.0000  19.0000
23399  Default  117.000000      military  117.000000  18.0000  18.0000
25451  Default  125.000000          near  125.000000  17.0000  17.0000
2781   Default  104.000000        attack  104.000000  16.0000  16.0000
13089  Default   90.000000    government   90.000000  15.0000  15.0000
36642  Default   59.000000         total   59.000000  14.0000  14.0000
7344   Default   62.000000     coalition   62.000000  13.0000  13.0000
28584  Default   62.000000           pro   62.000000  12.0000  12.0000
15883  Default   55.000000       involve   55.000000  11.0000  11.0000
30365  Default   58.000000    reportedly   58.000000  10.0000  10.0000
34424  Default   87.000000         stage   87.000000   9.0000   9.0000
36876  Default   57.000000         troop   57.000000   8.0000   8.0000
30184  Default   65.000000        regime   65.000000   7.0000   7.0000
32093  Default   50.000000      saturday   50.000000   6.0000   6.0000
6406   Default   77.000000      casualty   77.000000   5.0000   5.0000
36043  Default   47.000000           tfg   47.000000   4.0000   4.0000
28292  Default   52.000000      position   52.000000   3.0000   3.0000
28727  Default   48.000000      province   48.000000   2.0000   2.0000
5379   Default   51.000000        border   51.000000   1.0000   1.0000
...        ...         ...           ...         ...      ...      ...
39699  Topic10    3.077294          zone    3.081008   2.7266  -5.9216
27454  Topic10    2.938732     passenger    2.942446   2.7266  -5.9677
32229  Topic10    2.785892         scene    2.789606   2.7265  -6.0211
30667  Topic10    2.728252         river    2.731966   2.7265  -6.0420
26862  Topic10    2.721999        outage    2.725713   2.7265  -6.0443
13398  Topic10    2.719788        gunmen    2.723502   2.7265  -6.0451
17130  Topic10   13.823446          july   13.918732   2.7210  -4.4193
5294   Topic10   27.954958          bomb   31.917145   2.5953  -3.7151
14864  Topic10   18.158951         house   20.273251   2.6177  -4.1465
10981  Topic10    8.142507     explosive    9.785889   2.5440  -4.9485
15236  Topic10    6.755661           ied    8.028862   2.5552  -5.1353
6440   Topic10    7.819457         cause   10.830172   2.4021  -4.9890
30708  Topic10   17.138393          road   31.312456   2.1252  -4.2043
26623  Topic10   10.096383          open   17.019960   2.2056  -4.